# Real data comparison

In [1]:
%pylab inline

import scipy.sparse
import pandas as pd
from scipy.io import loadmat
from sklearn.datasets import make_sparse_spd_matrix
from sklearn.utils.extmath import squared_norm
from sklearn.covariance import empirical_covariance

from regain import datasets; reload(datasets)
from regain.datasets import is_pos_def, is_pos_semidef
from regain.plot import plot_graph_with_latent_variables
from regain.admm import latent_time_graph_lasso_; reload(latent_time_graph_lasso_)
from regain.admm import latent_time_graph_lasso_v2_; reload(latent_time_graph_lasso_v2_)
from regain.admm import latent_time_graph_lasso_v3_; reload(latent_time_graph_lasso_v3_)
from regain.admm import time_graph_lasso_; reload(time_graph_lasso_);
from regain.admm import latent_graph_lasso_; reload(latent_graph_lasso_);
from regain.utils import error_norm

Populating the interactive namespace from numpy and matplotlib


In [2]:
data = pd.read_csv("/home/fede/src/slipguru/regain/regain/data/finance.csv", header=0, index_col=0)

In [16]:
data.loc[:, ['A', 'AA']][10:12]

,A,AA
20090904,1.22,0.38
20090909,3.87,4.05


In [48]:
n = 30
data_list = np.array([data.loc[:,['AMZN', 'IBM', 'GOOG', 'MSFT','APOL']][i:i+n].values 
             for i in range(1, data.shape[0], n)][:-1])

In [60]:
[data.loc[:,['AMZN', 'IBM', 'GOOG', 'MSFT','APOL']][i:i+n] 
             for i in range(1, data.shape[0], n)][:-1]

[          AMZN   IBM  GOOG  MSFT  APOL
 20090824 -0.62 -0.52  0.72  0.91 -0.47
 20090825 -0.68 -0.73  0.25 -0.32  1.43
 20090826 -0.27  0.49 -0.76 -0.41  0.88
 20090827 -0.02 -0.42 -0.80  0.18  0.97
 20090828 -1.46 -0.63  0.10  0.34  0.92
 20090831 -1.39  0.36 -0.16  0.38 -3.52
 20090901 -0.55  0.80  0.68 -0.68  0.09
 20090902 -0.97 -0.19 -0.28 -0.26  1.37
 20090903 -0.28 -0.48  0.31  0.36  1.77
 20090904 -0.51 -0.06 -0.21  1.08 -1.50
 20090909  3.15 -1.72 -0.54 -0.47  0.49
 20090910  1.12 -0.06  0.66  0.05  0.16
 20090911  1.05  0.55  0.48 -0.33 -1.90
 20090914 -1.03  0.48  0.41  0.34  0.89
 20090915 -0.96 -0.19 -0.08  0.21  1.06
 20090916  7.44  0.95  1.13 -1.12 -1.50
 20090917 -0.21  0.13  0.78  0.48  2.79
 20090918 -0.55 -0.18 -0.42 -0.53 -0.73
 20090921  0.73 -0.02  1.55  0.58  8.71
 20090922  3.00 -0.49 -0.11  1.34 -5.10
 20090923 -0.63  0.18  0.71  0.59 -0.31
 20090924  0.13  0.52  0.08  1.32  0.42
 20090925 -1.29  0.55 -0.43 -1.07  0.63
 20090928  0.58 -2.73 -0.06 -0.19 -0.69


In [55]:
data_grid = data_list.transpose(1,2,0)

In [56]:
from sklearn.model_selection import GridSearchCV, ShuffleSplit
from regain import utils; reload(utils)
from regain.admm import time_graph_lasso_; reload(time_graph_lasso_)
from regain.admm import latent_time_graph_lasso_; reload(latent_time_graph_lasso_);
from regain.admm import latent_time_graph_lasso_v3_; reload(latent_time_graph_lasso_v3_);

mdl = latent_time_graph_lasso_v3_.LatentTimeGraphLasso(
    bypass_transpose=False, assume_centered=0, verbose=0, rtol=1e-5, tol=1e-5,
    max_iter=500, rho=1. / np.array(data_list).shape[1])

ltgl = GridSearchCV(mdl, dict(#tau=[0.25, 0.2, 0.1],
                             tau=np.linspace(0.01, 0.8, 20),
                             eta=[0], beta=[0],
                             #alpha=[0.05, 0.1, 1],
                             alpha=np.linspace(0.01, 0.2, 5)),
                    cv=ShuffleSplit(1, test_size=.5)).fit(data_grid)

In [58]:
ltgl.best_estimator_.precision_

array([[[ 0.2663739 , -0.        , -0.        , -0.        ,  0.        ],
        [-0.        ,  0.98324308, -0.        , -0.        ,  0.        ],
        [-0.        , -0.        ,  1.41211198, -0.        , -0.01293394],
        [-0.        , -0.        , -0.        ,  0.96868926, -0.        ],
        [ 0.        ,  0.        , -0.01293394, -0.        ,  0.17808471]],

       [[ 0.06496547, -0.        , -0.        , -0.09513111,  0.        ],
        [-0.        ,  0.58114476,  0.        ,  0.        ,  0.        ],
        [-0.        ,  0.        ,  0.49464878,  0.        , -0.        ],
        [-0.09513111,  0.        ,  0.        ,  0.53323127, -0.        ],
        [ 0.        ,  0.        , -0.        , -0.        ,  0.08861144]],

       [[ 0.27226079, -0.        , -0.        , -0.        ,  0.        ],
        [-0.        ,  1.10773215, -0.        , -0.        ,  0.        ],
        [-0.        , -0.        ,  0.72082331, -0.        ,  0.        ],
        [-0.        ,

In [ ]:
['AMZN', 'IBM', 'GOOG', 'MSFT','APOL']